In [1]:
CapacityMetricsWorkspace = 'Microsoft Fabric Capacity Metrics'
CapacityMetricsDataset = 'Fabric Capacity Metrics'

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 3, Finished, Available, Finished)

### Get the workspace id

In [2]:
workspaceId = spark.conf.get('trident.workspace.id')
print(f"{workspaceId=}")

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 4, Finished, Available, Finished)

workspaceId='81129f4c-646d-4dea-a61c-d4b293dbf555'


In [3]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}', headers=header)
workspaceName = response.json().get('displayName')
print(f'{workspaceName=}')

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 5, Finished, Available, Finished)

workspaceName='WS_DW_Query_Cost_Test_Take_Six'


### Create lakehouses

In [4]:
from notebookutils import mssparkutils

for lakehouseName in ['LH_SampleData', 'LH_QueryResults']:
    try:
        lakehouseItem = mssparkutils.lakehouse.create(lakehouseName, "", workspaceId)
    except Exception as e:
        # print(e)
        pass

LH_SampleData = mssparkutils.lakehouse.get("LH_SampleData", workspaceId)
LH_QueryResults = mssparkutils.lakehouse.get("LH_QueryResults", workspaceId)
print(f'{LH_SampleData=}')
print(f'{LH_QueryResults=}')

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 6, Finished, Available, Finished)

LH_SampleData={'id': 'de46dc8d-cdde-4900-bbfd-ecc2b2d16186', 'type': 'Lakehouse', 'displayName': 'LH_SampleData', 'description': '', 'workspaceId': '81129f4c-646d-4dea-a61c-d4b293dbf555', 'properties': {'abfsPath': 'abfss://81129f4c-646d-4dea-a61c-d4b293dbf555@onelake.dfs.fabric.microsoft.com/de46dc8d-cdde-4900-bbfd-ecc2b2d16186'}}
LH_QueryResults={'id': 'd3dc76f8-2afb-480c-a15c-6a4bf947ae24', 'type': 'Lakehouse', 'displayName': 'LH_QueryResults', 'description': '', 'workspaceId': '81129f4c-646d-4dea-a61c-d4b293dbf555', 'properties': {'abfsPath': 'abfss://81129f4c-646d-4dea-a61c-d4b293dbf555@onelake.dfs.fabric.microsoft.com/d3dc76f8-2afb-480c-a15c-6a4bf947ae24'}}


### Get notebook definition from github

In [5]:
import requests, json, base64
url = "https://raw.githubusercontent.com/bretamyers/FabricTools/dev/FabricDWQueryCostAnalyzer/NB_DW_Load_Cost_Analyzer.ipynb"
response = requests.get(url)

b = base64.b64encode(bytes(json.dumps(response.json()), 'utf-8')) # convert to bytes
base64_str = b.decode('utf-8') # convert bytes to string
notebookDefDict = json.loads(base64.b64decode(base64_str).decode('utf-8')) # convert base64 bytes to string and then to dictionary
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse'] = LH_QueryResults.get('id')
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse_name'] = LH_QueryResults.get('displayName')
notebookDefDict['metadata']['dependencies']['lakehouse']['default_lakehouse_workspace_id'] = LH_QueryResults.get('workspaceId')
notebookDefBase64 = base64.b64encode(json.dumps(notebookDefDict).encode('utf-8')).decode('utf-8') # convert dictionary back to base64 string

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 7, Finished, Available, Finished)

### Create notebook from the definition

In [6]:
import requests, json, time, base64

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

body = {
    "displayName": "NB_DW_Load_Cost_Analyzer",
    "description": "A notebook to run data warehouse queries and capture the duration and cost of those queries.",
    "definition": {
        "format": "ipynb",
        "parts": [
            {
                "path": "notebook-content.py"
                ,"payload": notebookDefBase64
                ,"payloadType": "InlineBase64"
            }
        ]
    }
}

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header, data=json.dumps(body))

if response.status_code == 202:
    time.sleep(int(response.headers.get('Retry-After')))
    for retry in range(5):
        responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
        print(responseStatus.text)
        print(responseStatus.headers)
        if responseStatus.headers.get('Location') is not None:
            break
        else:
            time.sleep(int(responseStatus.headers.get('Retry-After')))
else:
    print(f"Error - {response.json()}")

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 8, Finished, Available, Finished)

{"status":"Succeeded","createdTimeUtc":"2024-09-24T14:08:24.1378933","lastUpdatedTimeUtc":"2024-09-24T14:08:25.8722942","percentComplete":100,"error":null}
{'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Content-Length': '131', 'Content-Type': 'application/json; charset=utf-8', 'Content-Encoding': 'gzip', 'Location': 'https://api.fabric.microsoft.com/v1/operations/e530debc-70a5-40fb-9f83-fc4425e8ef1a/result', 'x-ms-operation-id': 'e530debc-70a5-40fb-9f83-fc4425e8ef1a', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': '37dd22b1-107f-495f-9cf8-84ae9e1ce9db', 'Access-Control-Expose-Headers': 'RequestId,Location,x-ms-operation-id', 'request-redirected': 'true', 'home-cluster-uri': 'https://wabi-west-us3-a-primary-redirect.analysis.windows.net/', 'Date': 'Tue, 24 Sep 2024 14:08:44 GMT'}


### Get the notebook id

In [7]:
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }
          
# Need to implement for pagination
response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/notebooks', headers=header)

for notebook in response.json().get('value'):
    if notebook.get('displayName') == 'NB_DW_Load_Cost_Analyzer':
        notebookId = notebook.get('id')
        print(f'{notebookId=}')


StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 9, Finished, Available, Finished)

notebookId='870b3457-6960-44bb-be75-37942a2d7e52'


### Create Warehouse

In [8]:
import requests, json, time

body = {"displayName": "WH_SampleData"}

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header, data=json.dumps(body))

if response.status_code == 202:
    print(f'Creating Warehouse {body.get("displayName")}')
    time.sleep(int(response.headers.get('Retry-After')))
    for retry in range(5):
        responseStatus = requests.request(method='get', url=response.headers.get('Location'), headers=header)
        print(responseStatus.text)
        print(responseStatus.headers)
        if responseStatus.json().get('status') != 'Succeeded':
            time.sleep(int(responseStatus.headers.get('Retry-After')))
        else:
            if responseStatus.headers.get('Location') is None: # no result was return but operation completed
                break
            else:
                responseResult = requests.request(method='get', url=responseStatus.headers.get('Location'), headers=header)
                print(f"Succeeded {responseResult.json()}")
                break
else:
    print(response.text)

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 10, Finished, Available, Finished)

Creating Warehouse WH_SampleData
{"status":"Succeeded","createdTimeUtc":"2024-09-24T14:08:48.9976506","lastUpdatedTimeUtc":"2024-09-24T14:09:05.7479145","percentComplete":100,"error":null}
{'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Content-Length': '133', 'Content-Type': 'application/json; charset=utf-8', 'Content-Encoding': 'gzip', 'Location': 'https://api.fabric.microsoft.com/v1/operations/7d9f401a-4fb8-40e4-bc42-82bc47513a95/result', 'x-ms-operation-id': '7d9f401a-4fb8-40e4-bc42-82bc47513a95', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': '12f169db-bb50-4b7b-9929-aae0a110a53b', 'Access-Control-Expose-Headers': 'RequestId,Location,x-ms-operation-id', 'request-redirected': 'true', 'home-cluster-uri': 'https://wabi-west-us3-a-primary-redirect.analysis.windows.net/', 'Date': 'Tue, 24 Sep 2024 14:09:09 GMT'}
Succeeded {'id': 'ac682e19-a4fe-4b58-964a-61df

In [9]:
import requests, json, time

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/warehouses', headers=header)

for warehouse in response.json().get('value'):
    if warehouse.get('displayName') == 'WH_SampleData':
        WH_SampleData = warehouse
        break
print(WH_SampleData)

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 11, Finished, Available, Finished)

{'id': 'ac682e19-a4fe-4b58-964a-61df890d3c32', 'type': 'Warehouse', 'displayName': 'WH_SampleData', 'description': '', 'workspaceId': '81129f4c-646d-4dea-a61c-d4b293dbf555', 'properties': {'connectionInfo': 'n52dzjnhphme5jslxpytuo62ni-jsprfalnmtve3jq42szjhw7vku.datawarehouse.fabric.microsoft.com', 'connectionString': 'n52dzjnhphme5jslxpytuo62ni-jsprfalnmtve3jq42szjhw7vku.datawarehouse.fabric.microsoft.com', 'createdDate': '2024-09-24T14:08:48.327', 'lastUpdatedTime': '2024-09-24T14:09:05.573'}}


### Load data to Lakehouse

In [10]:
import azure.storage.blob
import pandas as pd
from notebookutils import mssparkutils

spark.conf.set('spark.sql.parquet.int96RebaseModeInWrite', 'LEGACY')
spark.conf.set('spark.sql.parquet.datetimeRebaseModeInWrite', 'LEGACY')

account_url = "https://fabrictutorialdata.blob.core.windows.net"
blob_service_client = azure.storage.blob.BlobServiceClient(account_url)
container_client = blob_service_client.get_container_client('sampledata')

tableList = list()
for blob in container_client.walk_blobs(name_starts_with='WideWorldImportersDW/parquet/tables/', delimiter='/'):
    tableName = blob.name.split('/')[-1].split('.parquet')[0]
    tableList.append(tableName)

for table in tableList:
    print(f'Loading Table: {table}')
    if mssparkutils.fs.exists(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}"):
        mssparkutils.fs.rm(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}", recurse=True)

    for blob in container_client.walk_blobs(name_starts_with=f"WideWorldImportersDW/parquet/tables/", delimiter='/'):
        if f'{table}.parquet' == blob.name.split('/')[-1]:
            dfPandas = pd.read_parquet(f'{account_url}/sampledata/{blob.name}', engine='pyarrow', storage_options={'anon': True})

            if not dfPandas.empty:
                dfSpark = spark.createDataFrame(dfPandas)
                dfSpark.write.format('delta').mode('append').option('mergeSchema', "true").save(f"{LH_SampleData.get('properties').get('abfsPath')}/Tables/{table}")
            else:
                print('Bad file', blob.name)

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 12, Finished, Available, Finished)

Loading Table: DimCity
Loading Table: DimCustomer
Loading Table: DimDate
Loading Table: DimEmployee
Loading Table: DimPaymentMethod
Loading Table: DimStockItem
Loading Table: DimSupplier
Loading Table: DimTransactionType
Loading Table: FactMovement
Loading Table: FactOrder
Loading Table: FactPurchase
Loading Table: FactSale
Loading Table: FactStockHolding
Loading Table: FactTransaction


### Load data to Warehouse

In [11]:
from notebookutils import mssparkutils  
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType, DoubleType
import pyodbc, struct, itertools, time, datetime, re, uuid, json

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={WH_SampleData.get("properties").get("connectionString")};Database={WH_SampleData.get("displayName")}'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

def get_result_set(cursor):
    if cursor.description:
        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
    else:
        resultList = []
        resultColumns = []
    return [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    with conn.cursor() as cursor:
        while retry in range(5):
            cursor.execute(f"""SELECT TABLE_NAME FROM LH_SampleData.INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo'""")
            resultSet = get_result_set(cursor)
            if len(resultSet) > 0:
                break
            else:
                time.sleep(10)
        while cursor.nextset():
            queryMessage += ",".join([str(cursor.messages) if cursor.messages else ""])
            resultSet.append(get_result_set(cursor))
        cursor.commit()

    for table in resultSet:
        with conn.cursor() as cursor:
            query = f"""IF OBJECT_ID('dbo.{table.get("TABLE_NAME")}', 'U') IS NOT NULL DROP TABLE dbo.{table.get("TABLE_NAME")}; CREATE TABLE dbo.{table.get("TABLE_NAME")} AS SELECT * FROM LH_SampleData.dbo.{table.get("TABLE_NAME")}"""
            print(query)
            cursor.execute(query)
            cursor.commit()


StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 13, Finished, Available, Finished)

IF OBJECT_ID('dbo.DimCity', 'U') IS NOT NULL DROP TABLE dbo.DimCity; CREATE TABLE dbo.DimCity AS SELECT * FROM LH_SampleData.dbo.DimCity
IF OBJECT_ID('dbo.DimPaymentMethod', 'U') IS NOT NULL DROP TABLE dbo.DimPaymentMethod; CREATE TABLE dbo.DimPaymentMethod AS SELECT * FROM LH_SampleData.dbo.DimPaymentMethod
IF OBJECT_ID('dbo.DimDate', 'U') IS NOT NULL DROP TABLE dbo.DimDate; CREATE TABLE dbo.DimDate AS SELECT * FROM LH_SampleData.dbo.DimDate
IF OBJECT_ID('dbo.DimSupplier', 'U') IS NOT NULL DROP TABLE dbo.DimSupplier; CREATE TABLE dbo.DimSupplier AS SELECT * FROM LH_SampleData.dbo.DimSupplier
IF OBJECT_ID('dbo.FactMovement', 'U') IS NOT NULL DROP TABLE dbo.FactMovement; CREATE TABLE dbo.FactMovement AS SELECT * FROM LH_SampleData.dbo.FactMovement
IF OBJECT_ID('dbo.DimCustomer', 'U') IS NOT NULL DROP TABLE dbo.DimCustomer; CREATE TABLE dbo.DimCustomer AS SELECT * FROM LH_SampleData.dbo.DimCustomer
IF OBJECT_ID('dbo.DimStockItem', 'U') IS NOT NULL DROP TABLE dbo.DimStockItem; CREATE TABL

### Refresh Fabric Capacity Metrics App so that the latest workspace artifacts are in the model

In [44]:
import requests, json, time

def model_refresh():
    header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
            ,"Content-Type": "application/json"
            }

    response = requests.get('https://api.fabric.microsoft.com/v1/workspaces', headers=header)

    capacityWorkspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == CapacityMetricsWorkspace][0]
    print(f'{capacityWorkspaceId = }')

    response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets", headers=header)

    datasetId = [dataset.get('id') for dataset in response.json().get('value') if dataset.get('name') == CapacityMetricsDataset][0]
    print(f'{datasetId = }')

    tableList = [{"table": "Capacities"}
                ,{"table": "Workspaces"}
                ,{"table": "TimePoints"}
                ,{"table": "TimePointBackgroundDetail"}
                ,{"table": "Items"}
                ]
    body = {"objects": tableList} # Need to ask Pat what tables are import and what are direct query
    # response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes", headers=header, data=json.dumps(body))
    response = requests.post(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes", headers=header)

    refreshId = response.headers.get('RequestId')
    print(f'{refreshId = }')

    for attempt in range(12): 
        # https://learn.microsoft.com/en-us/power-bi/connect-data/asynchronous-refresh#get-refreshes
        response = requests.get(f"https://api.powerbi.com/v1.0/myorg/groups/{capacityWorkspaceId}/datasets/{datasetId}/refreshes?$top=1", headers=header)
        if response.json().get('value')[0].get('status') != 'Unknown':
            print(f'Refresh Complete')
            break
        else:
            print(f'Refreshing tables ...')
            time.sleep(10)

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 46, Finished, Available, Finished)

In [46]:

import requests, math

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

body = {
  "queries": [
    {
      "query": f"""
        DEFINE
            VAR __DS0Core = 
                SUMMARIZE('Workspaces', 'Workspaces'[WorkspaceName], 'Workspaces'[WorkspaceId])
            VAR __DS0PrimaryWindowed = 
                TOPN(501, __DS0Core, 'Workspaces'[WorkspaceName], 1, 'Workspaces'[WorkspaceId], 1)
        EVALUATE
            __DS0PrimaryWindowed
    """
    }
  ]
}

response = requests.post(f'https://api.powerbi.com/v1.0/myorg/datasets/{datasetId}/executeQueries', headers=header, json=body )

for retry in range(10):
    model_refresh()
    workspaceFound = False
    for results in response.json().get('results'):
        for table in results.get('tables'):
            for row in table.get('rows'):
                print(row)
                if row.get('Workspaces[WorkspaceName]') == 'WS_DW_Query_Cost_Test_Take_Seven':
                    workspaceFound = True
    if workspaceFound:
        print('Workspace was found in the capacity metrics app model')
        break
    else:
        print('Workspace was not found. Refreshing the model...')
        time.sleep(180)

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 48, Finished, Available, Finished)

capacityWorkspaceId = '0315286f-829f-410f-a0aa-1e45af27297c'
datasetId = 'a3443502-d96b-4b3f-99e5-76920052edad'
refreshId = '7b1039c1-2cb4-47dd-aee4-e4d2f7529c67'
Refresh Complete
{'Workspaces[WorkspaceName]': 'WS_Temp', 'Workspaces[WorkspaceId]': '4D0C3D99-D190-42D4-B844-973CA02D1B7C'}
{'Workspaces[WorkspaceName]': 'WS_Demo_Inventory', 'Workspaces[WorkspaceId]': '55B76DF6-9F8B-47BB-A23B-2D8B29F21C1F'}
{'Workspaces[WorkspaceName]': 'WS_DW_Query_Cost', 'Workspaces[WorkspaceId]': '93D2465A-1E27-4AC4-8DD7-CF80B0B88331'}
{'Workspaces[WorkspaceName]': 'WS_EventStream_Trigger_Workaround', 'Workspaces[WorkspaceId]': 'C7D2DB92-860B-47D7-BF85-8A0275CFC191'}
{'Workspaces[WorkspaceName]': 'WS_DW_Query_Cost_Test_Take_Three', 'Workspaces[WorkspaceId]': '0F73EEDA-4D74-41D2-8480-06A358132BDF'}
{'Workspaces[WorkspaceName]': 'WS_DW_Query_Cost_Test_Take_Two', 'Workspaces[WorkspaceId]': '63D505F2-D635-4093-B255-B418710B50E8'}
{'Workspaces[WorkspaceName]': 'WS_DW_Query_Cost_Test', 'Workspaces[WorkspaceId]

### Run the notebook

In [14]:
import requests, json, time

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
          }

body = {
    "executionData": {
        "parameters": {
            "FabricDWWorkspaceName": {"value": workspaceName, "type": "string"}
            ,"FabricDWName": {"value": "WH_SampleData", "type": "string"}
            ,"ConcurrencyNum": {"value": "1", "type": "int"}
            ,"CapacityMetricsWorkspace": {"value": CapacityMetricsWorkspace, "type": "string"}
            ,"CapacityMetricsDataset": {"value": CapacityMetricsDataset, "type": "string"}
        }
    }
}

# print(json.dumps(body, indent=4))

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{notebookId}/jobs/instances?jobType=RunNotebook', headers=header, data=json.dumps(body))

if response.status_code == 202:
    print('Started Execution of Notebook')
    time.sleep(10) # Sleep for 10 seconds to wait for job to get started. TODO Change in the future to a retry
    print(f"{response.headers=}")
    # jobId = response.headers.get('Location')
    while True:
        # print(f"{response.headers.get('Location')=}")
        responseStatus = requests.request(method="get", url=response.headers.get('Location'), headers=header)
        print(f"Status: {responseStatus.json().get('status')} | {responseStatus.json().get('failureReason') if responseStatus.json().get('failureReason') != None else ''}")
        if responseStatus.json().get('status') in ['Succeeded', 'Failed', 'Completed']:
            print(responseStatus.json().get('status'))
            break
        else:
            time.sleep(120) # Wait 2 minutes to check the status of the run
else:
    print('Error trying to execute')

StatementMeta(, f69fa1db-6507-4564-952c-81a1af2465a5, 16, Submitted, Running, Running)

Started Execution of Notebook
response.headers={'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Content-Length': '0', 'Content-Type': 'application/octet-stream', 'Content-Encoding': 'gzip', 'Location': 'https://api.fabric.microsoft.com/v1/workspaces/81129f4c-646d-4dea-a61c-d4b293dbf555/items/870b3457-6960-44bb-be75-37942a2d7e52/jobs/instances/5e91b08a-28ed-4ff1-a04d-371915d97de4', 'Retry-After': '60', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': '74292d17-699c-4407-ae8f-37252e3e0cf4', 'Access-Control-Expose-Headers': 'RequestId,Location,Retry-After', 'request-redirected': 'true', 'home-cluster-uri': 'https://wabi-west-us3-a-primary-redirect.analysis.windows.net/', 'Date': 'Tue, 24 Sep 2024 14:11:15 GMT'}
Status: InProgress | 
Status: InProgress | 
Status: InProgress | 
Status: InProgress | 
Status: InProgress | 
Status: InProgress | 
Status: InProgress | 
S

In [ ]:
# import requests

# response = requests.request(method="get", url=f'https://api.fabric.microsoft.com/v1/workspaces/a3b40eca-afde-448e-bc54-af3d318657b0/items/d20ea153-dbf1-4821-8243-e823bf95bdbb/jobs/instances/237f073e-f403-4d12-a2ae-63377fbe0335', headers=header)
# print(response.status_code)
# print(response.json())


StatementMeta(, , , Cancelled, , Cancelled)